# Introduction

I made this notebook because I wanted to see how to measure the correlation between different types of variables. I will consider two types of variables: categorical and continuous. As a really simple definition, we'll say that a categorical variable can have only have one of several values (e.g. 'A', 'B', 'C' or 1, 2, 3) but a categorical variable can have arbitrary (floating point) values. For example, in the March Tabular Playground competition we have 19 categorical features (which contain string values), 11 continuous features and the target is categorical (0 or 1). It is important to know how strongly the target depends on the features, and also how strong each features depends on the others. This information is helpful for example when deciding which features to drop, if it's a good idea to try feature engineering with PCA.

There are three possibilities when investigating two features: both are categorial, both are continuous or one is catgorical and the other is continuous. After some searching, I found [this article by Outside Two Standard Deviations](https://medium.com/@outside2SDs/an-overview-of-correlation-measures-between-categorical-and-continuous-variables-4c7f85610365) that outlines some statistical methods that can be used for these cases, the take-away is this:

- 2 categorical features: use Lambda or Corrected Cramer's V
- 1 categorical & 1 continuous feature: use Point Biserial or Logistic Regression
- 2 continuous features: use Spearman, Kendall or Pearson correlation (where Spearman seems to be the most useful in general)

I will go into some of the details and apply these methods to a toy dataset. Then I will analyse the competition dataset.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import random
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
sns.set(palette='Set2')

# Generate toy dataset

I will make a dataframe with continuous and categorical variables which are related to a target column in various ways:

- The target itself is 0 (75 % of rows) or 1
- `cat0` and `cat1` are strongly correlated with the target
- `cat2` is partially correlated with the target
- `cat3` is uncorrelated with the target
- `cont0` is negatively correlated with the target
- `cont1` is partially correlated with the target
- `cont2` is uncorrelated with the target
- `cont3` is partially correlated with cont2

In [ ]:
random.seed(0)
ROWS = 100
df_toy = pd.DataFrame(dict(target=np.zeros(ROWS)))
df_toy.target.iloc[:75] = 0
df_toy.target.iloc[75:] = 1
df_toy['cat0'] = df_toy.target.apply(lambda x: 'A' if x else 'B')
df_toy['cat1'] = df_toy.target.apply(lambda x: random.choice(['A', 'B']) if int(x) else random.choice(['C', 'D']))
df_toy['cat2'] = df_toy.target.apply(lambda x: random.choice(['A', 'B']) if int(x) else random.choice(['B', 'C']))
df_toy['cat3'] = df_toy.target.apply(lambda x: random.choice(['A', 'B']))
df_toy['cont0'] = df_toy.target.apply(lambda x: -3*int(x))
df_toy['cont1'] = df_toy.target.apply(lambda x: int(x) + random.gauss(0, 1))
df_toy['cont2'] = np.random.rand(ROWS)
df_toy['cont3'] = 2 * df_toy.cont2 + np.random.rand(ROWS)
df_toy = df_toy.sample(frac=1)   # shuffle dataset
display(df_toy.sample(10))

## Visualize toy dataset

We'll make some quick plots to check that the relationships are as intended.

### Categorical features

In [ ]:
cat_features = ['cat0', 'cat1', 'cat2', 'cat3']
df_toy_cat = df_toy.melt(value_vars=cat_features, id_vars=['target']).sort_values(['variable', 'value'])
sns.catplot(data=df_toy_cat, x='value', y='target', col='variable', col_wrap=2, sharex=False)

### Continuous features vs target

In [ ]:
cont_features = ['cont0', 'cont1', 'cont2', 'cont3']
df_toy_cont = df_toy.melt(value_vars=cont_features, id_vars=['target']).sort_values(['variable', 'value'])
sns.relplot(data=df_toy_cont, x='value', y='target', col='variable', col_wrap=2)

### cont2 vs cont3

In [ ]:
sns.relplot(data=df_toy, x='cont2', y='cont3', hue='target')

# Analyze toy dataset

Now that we have a toy dataset where we know the relationships between the features and the target, we can use it to test and understand the statistical metrics for correlations mentionend erlier.

## Correlation of two continuous features

We'll start with the easiest case. For continuous features, pandas provides the `corr` method, which I've seen in plenty of Kaggle notebooks. Pandas provides the Pearson, Kendall Tau or Spearman correlation, and Pearson is the default. It's easy  to make a correlation heatmap using `corr`. When comparing the different correlation methods, we can observe that the partial correlations are assigned a lower value when using Kendal Tau, but overall, the gist is the same for all three methods.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
plt.subplots_adjust(wspace=0.3)

for method, ax in zip(['pearson', 'kendall', 'spearman'], axs.flatten()):
    corr = df_toy.corr(method=method)
    mask = np.triu(np.ones_like(corr, dtype=np.bool))
    # plot heatmap
    sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='coolwarm',
                cbar_kws={"shrink": .8}, ax=ax)
    # yticks
    ax.tick_params(labelrotation=0);
    ax.title.set_text(f'Correlation method: {method.capitalize()}')
    
fig.delaxes(axs.flatten()[-1])

We can observe the following:

- Although the target is categorical, it has been included by pandas because its data type is numerical.
- As expected, `target` is perfectly correlated with `cont0` and somewhat correlated with `cont1`.
- We also see the expected correlation between `cont2` and `cont3`.

## Correlation of two categorical features

We'll use [Cramer's V](https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V) to make a correlation matrix similar to the one for categorical features. This metric is essentially $\chi^2$ which has been scaled to be between zero and one. It is calculated like this:
$$
    V = \sqrt{\frac{\chi^2 / n}{\min(k-1, r-1)}}
$$
where n is the number of observations, k is the number of columns and r is the number of rows. Just like for the categorical variables, a value of 0 means no correlation, and a value of one means perfect correlation. The matrix we get fits the expectations stated above quite nicely. There is a bias correction available, and when it is turned on, the values for the uncorrelated features (e.g. cat2 and target) are somewhat closer to zero, while the more strongly correlated features are not affected much. Note, that we're cheating a little bit here: we're not really calculating the main diagonal, it is just set to 1.

In [ ]:
def cramer_v(df, feature1, feature2, bias_correction=True):
    tab = pd.crosstab(df[feature1], df[feature2])
    chi2 = stats.chi2_contingency(tab)[0]
    n = tab.to_numpy().sum()
    phi2 = chi2 / n
    k, r = tab.shape
    if bias_correction:
        phi2 = max(0, phi2 - (k - 1) * (r - 1) / (n - 1))
        k = k - (k - 1)**2 / (n - 1)
        r = r - (r - 1)**2 / (n - 1)
    v = np.sqrt(phi2 / min(k-1, r-1))
    return(v)

def cramer_matrix(df, features, bias_correction=True):
    n = len(features)
    data = np.ones([n, n])
    for i, j in itertools.combinations(range(n), 2):
        data[i, j] = cramer_v(df, features[i], features[j], bias_correction)
        data[j, i] = data[i, j]
    df_cramer = pd.DataFrame(data, index=features, columns=features)
    return df_cramer

fig, axs = plt.subplots(1, 2, figsize=(14, 7))

df_cramer = cramer_matrix(df_toy, cat_features + ['target'], False)
sns.heatmap(df_cramer, annot=True, fmt=".2f", cmap='coolwarm', cbar_kws={"shrink": .8}, ax=axs[0])
axs[0].set_title('Cramer`s V without bias correction')

df_cramer = cramer_matrix(df_toy, cat_features + ['target'], True)
sns.heatmap(df_cramer, annot=True, fmt=".2f", cmap='coolwarm', cbar_kws={"shrink": .8}, ax=axs[1])
axs[1].set_title('Cramer`s V with bias correction');

## Best categorical values to predict target

Cramer's V looks at all values of a categorical feature, but what happens if there is a value that is better at predicting the target than the others? For example, the correlation between cat2 and the target is only 66%, but we know that all "A" values correspond to a target value of 1, while all "C" values correspond to a target value of zero - only value "B" does not tell us anything about the target.

In [ ]:
sns.histplot(data=df_toy, x='cat2', hue='target', stat='probability', multiple='dodge')

As a starting point, we'll arrange our data by the values of every feature and see how strongly it will predict the target. In this case, an average value of the target of 0 or 1 indicates the best predicion power, while an average of 0.25 means this value tells us nothing about the target.

For `cat2`, the table below reveals the information that is also shown in the bar plot above, namely that if values 'A' or 'C' are encountered, a perfect prediction for the target value can be made, while value 'B' does not reveal anything about the target. This information cannot be extracted from Cramer's V.

In [ ]:
df_toy_melt = df_toy.melt(value_vars=['cat0', 'cat1', 'cat2', 'cat3'], id_vars='target')
df_toy_pred_power = df_toy_melt.groupby(['variable', 'value']).mean('target')
df_toy_pred_power.style.background_gradient(cmap='coolwarm')

Let's try to generalize this idea. The Gini Impurity seems like a good way to achieve this. It is calculated like this:
$$
    I_G = 1 - \sum_{i} \left(\frac{n_i}{N}\right)^2
$$
where $n_i$ are the counts four category $i$ and $N$ is the total number of observations. For example, if we have 5 observations where the target is 0, and 15 where it is 1, the impurity would be
$$
    I_G = 1 - \left(\frac{5}{20}\right)^2 - \left(\frac{15}{20}\right)^2 = 0.375
$$

We can take all observations for a given categorical variable and value, and calculate the impurity of the target values. The lowest possible value of 0 means that all target values are identical (i.e. the distribution 100% pure), higher values imply more of a mixture.

In [ ]:
def prediction_power_gini(df, value_vars=None, id_var='target'):
    df_res = df.melt(
        value_vars=value_vars, id_vars=id_var
    ).groupby(['variable', 'value']).size().reset_index().drop(0, axis=1)
    def gini(variable, value):
        df_gini = df[df[variable] == value][id_var].value_counts()
        impurity = 1
        for val in df_gini:
            impurity -= (val / df_gini.sum())**2
        return impurity
    df_res['gini'] = df_res.apply(lambda x: gini(x['variable'], x['value']), axis=1)
    return df_res

df_toy_pred_gini = prediction_power_gini(df_toy, value_vars=['cat0', 'cat1', 'cat2', 'cat3'], id_var='target')
display(df_toy_pred_gini.style.background_gradient(cmap='coolwarm_r'))

## Correlation between a continuous & a categorical feature

This is the last combination that we haven't discussed yet. Although the target has been included with the correlation of the continuous features above (and it works in this case), it is actually a categorical variable, so I would like to try a technique a more appropriate technique for this case. The most straight forward way seems to be point biserial correlation, so let's give it a try, there is a corresponding function available in `scipy.stats`. Note, that method only works for a binary categorical variable. This is fine for the target, but we cannot use it to find the correlations e.g. between cont0 and cat1.

The values we get are essentiale the same ones as for Spearman or Pearson correlation - so perhaps in general it would not be worth to spend time on biserial correlation, at least not in the context of machine learning.

In [ ]:
biserial = [stats.pointbiserialr(df_toy['target'], df_toy[feature])[0] for feature in cont_features]
df_biserial = pd.DataFrame(dict(target_correlation=biserial), index=cont_features)
df_biserial.style.background_gradient(cmap='coolwarm')

# Analyze the competition dataset

## Read in the data files

We'll only use the train data here, since the test data does not contain the target.

In [ ]:
input_path = Path('/kaggle/input/tabular-playground-series-mar-2021/')
df_train = pd.read_csv(input_path / 'train.csv', index_col='id')
display(df_train.head())

cat_features = list(df_train.drop('target', axis=1).select_dtypes(include=['object', 'category']).columns)
cont_features = list(df_train.drop('target', axis=1).select_dtypes(exclude=['object', 'category']).columns)
print(f'{len(df_train)} rows in train set, {len(cat_features)} categorical features, {len(cont_features)} continuous features')

## Correlation between categorical features

As above, we'll use Cramer's V to check out the correlations among the categorical features and the target.

In [ ]:
plt.figure(figsize=(16, 16))
df_cramer = cramer_matrix(df_train, cat_features + ['target'])
sns.heatmap(df_cramer, annot=True, fmt=".2f", cmap='coolwarm', cbar_kws={"shrink": .8})
plt.title('Cramer`s V')
plt.yticks(rotation=0);

We see that cat15, cat16 and cat18 have the strongest correlation with the target.

## Digging deeper into categorical features

Let's check if we can find the categorical values which are the best predictors for the target. We'll use the Gini impurity that was introduced for the toy dataset. We can make a simple plot of the impurity vs the variable. We see, that there is more or less the entire range from 0 to 1 for every variable. Many of the values with low impurity are extremely rare, so they are very useful for modeling. But we there are a few values that look interesting.

In [ ]:
def count_values(row):
    return len(df_train[df_train[row['variable']] == row['value']])

df_train_pred_power = prediction_power_gini(df_train, value_vars=cat_features, id_var='target').reset_index()
df_train_pred_power['counts'] = df_train_pred_power.apply(count_values, axis=1)
display(df_train_pred_power.head())
display(sns.catplot(data=df_train_pred_power, y='variable', x='gini'))
display(sns.relplot(data=df_train_pred_power, hue='variable', y='gini', x='counts'))

We can filter out the points with few counts and high impurities to get a better look at the interesting values in the dataset. I think this is a strong case to not just ignore cat10, since it has some promising categories.

In [ ]:
df_pred_power_filtered = df_train_pred_power[(df_train_pred_power.gini < 0.25) & (df_train_pred_power.counts > 1000)]
display(df_pred_power_filtered.sort_values('gini'))
sns.relplot(data=df_pred_power_filtered, hue='variable', y='gini', x='counts')

# Correlations between continuous features

As we saw with the toy dataset, we can get away with using the pandas `corr` method for measuring correlation of continuous features with a bivariate target variable. We'll compare how the three different methods perform on the competition dataset.

But first, we'll visualize the distribution of the continuous features.

In [ ]:
sns.displot(data=df_train[cont_features].melt(), x='value', col='variable', col_wrap=3)

These are obviously not normal distributions, however, this may have an effect when calculating correlations (see below). We can perform a quantile transformation to normalize the features.

In [ ]:
from sklearn.preprocessing import QuantileTransformer

df_train_norm = df_train.copy()
for feature in cont_features:
    df_train_norm[feature] = QuantileTransformer(
        output_distribution='normal').fit_transform(df_train_norm[feature].values.reshape(-1, 1))

sns.displot(data=df_train_norm[cont_features].melt(), x='value', col='variable', col_wrap=3)

Now, let's see how big the difference is between the 3 correlation methods, and between normalized and not-normalized data:

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(20, 30))
plt.subplots_adjust(wspace=0.3)

methods = ['pearson', 'kendall', 'spearman']
datasets = [('Not normalized', df_train), ('Normalized', df_train_norm)]
for i in range(3):
    for j in range(2):
        method = methods[i]
        ax = axs[i, j]
        df_label, df = datasets[j]
        corr = df.corr(method=method)
        mask = np.triu(np.ones_like(corr, dtype=np.bool))
        # plot heatmap
        sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='coolwarm',
                    cbar_kws={"shrink": .8}, ax=ax)
        # yticks
        ax.tick_params(labelrotation=0);
        ax.title.set_text(f'{df_label}, correlation method: {method.capitalize()}')

### Conclusion:

The only significant difference between the correlations when using normalized vs non-normalized data happens for Pearson. Kendall & Spearman are not affected by normalizing, but the values for Kendall are somewhat smaller than the other two methods. Given that Spearman correlation is very similar to Pearson with normalized data, my personal take-away is this: **Forget about normalization and just use Spearman, avoid Pearson & Kendall.** In any case, even though the values are different, all of the above heat-maps tell a similar story. In the context of machine learning, the normalization and correlation method probably doesn't matter that much.